# Data Mining and Exploration (INFR 11007)

## First Steps in EDA

### Numerical data description

In [1]:
import numpy as np
import pandas as pd

In [2]:
set1 = np.array([0, 1, 1, 1, 2, 3, 4, 4, 5, 9])
set2 = np.array([0, 1, 1, 1, 2, 3, 4, 4, 5, 9000])
print('Set 1: {}'.format(', '.join(list(map(str, set1)))))
print('Set 2: {}'.format(', '.join(list(map(str, set2)))))

Set 1: 0, 1, 1, 1, 2, 3, 4, 4, 5, 9
Set 2: 0, 1, 1, 1, 2, 3, 4, 4, 5, 9000


#### Location

**Not-robust measures:**
- Sample mean: $\displaystyle \bar x = \frac{1}{n} \sum_{i=1}^n x_i$
    - Estimator of the mean of r.v. $X$
    
**Robust measures:**
- Median: $\displaystyle \mathrm{median}(x) = \begin{cases}
    x_{(\frac{n+1}{2})} & \text{if } n \text{ is odd} \\
    \frac{1}{2} \left( x_{(\frac{n}{2})} + x_{(\frac{n}{2}+1)} \right) & \text{if } n \text{ is even}
\end{cases}$
- Mode: the value that occurs most frequently
- Quantile: $q_\alpha \approx x_{(\lceil n\alpha \rceil)}$
    - Quartile: $Q_1 = q_{0.25}$, $Q_2 = q_{0.5}$, $Q_3 = q_{0.75}$

In [3]:
d = {
    'mean': [set1.mean(), set2.mean()], 
    'median': [np.median(set1), np.median(set2)], 
    '$Q_1$': [np.quantile(set1, 0.25), np.quantile(set2, 0.25)], 
    '$Q_2$': [np.quantile(set1, 0.5), np.quantile(set2, 0.5)], 
    '$Q_3$': [np.quantile(set1, 0.75), np.quantile(set2, 0.75)]
}
pd.DataFrame(data=d, index=['Set 1','Set 2'])

,mean,median,$Q_1$,$Q_2$,$Q_3$
Set 1,3.0,2.5,1.0,2.5,4.0
Set 2,902.1,2.5,1.0,2.5,4.0


#### Scale

**Not-robust measures:**
- Sample variance: $\displaystyle \mathrm{Var}(x) = \frac{1}{n} \sum_{i=1}^n (x_i - \bar x)^2$
    - Estimator of the variance of r.v. $X$
- Sample standard deviation: $\mathrm{std}(x) = \sqrt{\mathrm{Var}(x)}$

**Robust measures:**
- Median absolute deviation: $\mathrm{MAD}(x) = \mathrm{median}(|x_i - \mathrm{median}(x)|)$
- Interquartile range: $\mathrm{IQR} = Q_3 - Q_1$

In [4]:
from scipy.stats import iqr

def mad(x):
    return np.median(np.abs(x - np.median(x)))

In [5]:
d = {
    'variance': [set1.var(), set2.var()], 
    'std': [set1.std(), set2.std()], 
    'MAD': [mad(set1), mad(set2)], 
    'IQR': [iqr(set1), iqr(set2)]
}
pd.DataFrame(data=d, index=['Set 1','Set 2'])

,variance,std,MAD,IQR
Set 1,6.40,2.529822,1.5,3.0
Set 2,7286222.89,2699.300445,1.5,3.0


#### Shape

**Not-robust measures:**
- Sample skewness: $\displaystyle \mathrm{skew}(x) = \frac{1}{n} \sum_{i=1}^n \left( \frac{x_i - \bar x}{\mathrm{std}(x)} \right)^3$
    - Location and scale are not taken into account
- Sample kurtosis: $\displaystyle \mathrm{kurt}(x) = \frac{1}{n} \sum_{i=1}^n \left( \frac{x_i - \bar x}{\mathrm{std}(x)} \right)^4$

**Robust measures:**
- Galton’s measure of skewness: $\displaystyle \frac{(Q_3 - Q_2) - (Q_2 - Q_1)}{Q_3 - Q_1}$
- Robust kurtosis: $\displaystyle \frac{(q_{7/8} - q_{5/8}) + (q_{3/8} - q_{1/8})}{Q_3 - Q_1}$

In [6]:
from scipy.stats import skew, kurtosis

def Galton(x):
    q1 = np.quantile(x, 0.25)
    q2 = np.quantile(x, 0.5)
    q3 = np.quantile(x, 0.75)
    return ((q3-q2)-(q2-q1))/(q3-q1)

def robust_kurt(x):
    q1 = np.quantile(x, 1/8)
    q2 = np.quantile(x, 1/4)
    q3 = np.quantile(x, 3/8)
    q5 = np.quantile(x, 5/8)
    q6 = np.quantile(x, 3/4)
    q7 = np.quantile(x, 7/8)
    return ((q7-q5)+(q3-q1))/(q6-q2)

In [7]:
d = {
    'skewness': [skew(set1), skew(set2)], 
    'Galton': [Galton(set1), Galton(set2)], 
    'kurtosis': [kurtosis(set1), kurtosis(set2)],
    'robustKurt': [robust_kurt(set1), robust_kurt(set2)]
}
pd.DataFrame(data=d, index=['Set 1','Set 2'])

,skewness,Galton,kurtosis,robustKurt
Set 1,1.074680,0.0,0.525391,0.541667
Set 2,2.666665,0.0,5.111106,0.541667


#### Multivariate measures

**Not-robust measures:**
- Sample covariance: $\displaystyle \mathrm{cov}(x,y) = \frac{1}{n} \sum_{i=1}^n (x_i - \bar x) (y_i - \bar y)$
    - Estimator of the covariance of r.v.s $X$ and $Y$
- Pearson's correlation coefficient: $\displaystyle \rho(x, y) = \frac{\mathrm{cov}(x,y)}{\mathrm{std}(x) \mathrm{std}(y)}$
    - $-1 \leq \rho \leq 1$
    - Aka linear correlation coefficient because linear relation is measured. 
    - Nonlinear: $\displaystyle \rho(g(x), g(y)) = \frac{\mathrm{cov}(g(x),g(y))}{\mathrm{std}(g(x)) \mathrm{std}(g(y))}$
- Sample covariance matrix
    - Positive semi-definite
    - Total variance $\displaystyle \sum_{i=1}^d \mathrm{Var}(x_i) = \sum_{i=1}^d \lambda_i$ (by eigenvalue decomopsition $\mathrm{Cov}(\mathbf x) = \mathbf U \boldsymbol\Lambda \mathbf U^\top$)
- Sample correlation matrix: $\displaystyle \rho(\mathbf x) = \mathrm{diag} \left( \frac{1}{\mathrm{std}(\mathbf x)} \right) \mathrm{Cov}(\mathbf x) \mathrm{diag} \left( \frac{1}{\mathrm{std}(\mathbf x)} \right)$

**Robust measures:**
- Kendall's $\tau$: $\displaystyle \tau(x,y) = \frac{n_c(x,y) - n_d(x,y)}{n(n-1)/2}$
    - $n_c(x,y)$ is the number of concordant pairs, i.e. $\#\left\{(x_i, y_i) \text{ and } (x_j, y_j), \forall i \neq j\right\}$ s.t. $\displaystyle \begin{cases}
x_i > x_j \\ 
y_i > y_j
\end{cases} \text{ or } \begin{cases}
x_i < x_j \\
y_i < y_j
\end{cases}$
    - $n_d(x,y)$ is the number of discordant pairs, i.e. $\#\left\{(x_i, y_i) \text{ and } (x_j, y_j), \forall i \neq j\right\}$ s.t. $\displaystyle \begin{cases}
x_i > x_j \\ 
y_i < y_j
\end{cases} \text{ or } \begin{cases}
x_i < x_j \\
y_i > y_j
\end{cases}$
    - If $\begin{cases}
x_i = x_j \\
y_i = y_j
\end{cases}$, the pair is neither concordant nor discordant

### Data visualisation

#### Bar plot

- Number of occurences of an attribute
- More useful to show relevant frequencies 

#### Box plot

- Based on robust measures (quartiles)

<embed src="boxplot.pdf" type="application/pdf" width="500px" height="520px"/>

#### Scatter plot

#### Histogram

$$B_i = [L+(i-1)h, L+ih) \quad i = 1, \ldots, k$$

- To visualise whold dataset, $L \leq \min (x_1, \ldots, x_n)$ and $L+kh \geq \max (x_1, \ldots, x_n)$
- Different starting values lead to differently looking histograms

#### Kernel density plot

$$\hat p(x) = \frac{1}{n} \sum_{i=1}^n K_h(x - x_i)$$

- Kernel: $\displaystyle \int K_h(x) \mathrm dx = 1$
- Bandwidth $h$: $\displaystyle K_h(x) = \frac{1}{h}K(\frac{1}{h})$
- Gaussian kernel: $\displaystyle K_h (\xi) = \frac{1}{\sqrt{2 \pi h^2}} \exp \left( -\frac{\xi^2}{2h^2} \right)$

#### Violin plot

- Combination of [box plot](#Box-plot) (robust) and [kernel density plot](#Kernel-density-plot) (non-robust but informative)

In [8]:
import seaborn as sns
sns.set()
iris = sns.load_dataset("iris")
ax = sns.violinplot(x="species", y="sepal_length", data=iris)

### Data pre-processing

#### Standardisation

**Centring matrix**

$$\tilde {\mathbf X} = \mathbf{X C}_n$$
- $\mathbf C_n = \mathbf I_n - \frac{1}{n} \boldsymbol 1_n \boldsymbol 1_n^\top$
- $\mathbf C_n \mathbf C_n = \mathbf C_n$
- Multiply from the left instead removes the sample mean of each column

**Scaling to unit variance**

$$\mathbf z_i = \mathrm{diag} \left( \frac{1}{\mathrm{std}(\mathbf x)} \right) \tilde{\mathbf x}_i$$

#### Outlier

- Turkey's fences $$[Q_1 - k(Q_3-Q_1), Q_3 + k(Q_3-Q_1)] = [Q_1 - k \mathrm{IQR}(x), Q_3 + k \mathrm{IQR}(x)]$$
    - $k \geq 0$ (most commonly $k=1.5$)
    - Used in [box plots](#Box-plot)

## Principal Component Analysis

### Variance maximisation

#### Sequtial

\begin{align*}
\text{First PC: } & \begin{cases}
\underset{\mathbf w_1}{\text{maximise}} & \mathrm{Var}(z_1) = \mathrm{Var}(\mathbf w_1^\top \mathbf x) = \mathbf w_1^\top \boldsymbol\Sigma \mathbf w_1 \\
\text{subject to} & \| \mathbf w_1 \| = 1
\end{cases} \\
\text{Subsequent PCs: } & \begin{cases}
\underset{\mathbf w_m}{\text{maximise}} & \mathbf w_m^\top \boldsymbol\Sigma \mathbf w_m \\
\text{subject to} & \| \mathbf w_m \| = 1 \\
& \mathbf w_m^\top \mathbf w_1 = 0 \quad i = 1, \ldots, m-1
\end{cases}
\end{align*}

1. Let $\mathbf w_1 = \mathbf{Ua}$, then $\displaystyle \mathbf w_1^\top \boldsymbol\Sigma \mathbf w_1 = \sum_{i=1}^d a_i^2 \lambda_i$ and $\displaystyle \| \mathbf w_1 \| = \sum_{i=1}^d a_i^2 = 1$
1. $\mathbf a = (1, 0, \ldots, 0)^\top$ is the solution of the optimisation problem if $\lambda_1 > \lambda_i$
1. $\mathbf w_1 = \mathbf u_1 = \mathbf{Ue}_1$ is the first PC direction, with variance $\lambda_1$
1. For subsequent calculations, constraints $a_i=0 \ (i=1,\ldots,m-1)$ exist


- PCs (scores) uncorrelated: 
\begin{align*}
\mathbb E [z_i z_j] &= \mathbb E [\mathbf w_i^\top \mathbf{xx}^\top \mathbf w_j] \\
&= \mathbf w_i^\top \boldsymbol\Sigma \mathbf w_j \\
&= \mathbf e_i^\top \mathbf U^\top \mathbf U \boldsymbol\Lambda \mathbf U^\top \mathbf{Ue}_j \\
&= 0
\end{align*}
- Fraction of variance explained: $\displaystyle \frac{\sum_{i=1}^k \lambda_i}{\sum_{i=1}^d \lambda_i}$

#### Simultaneous

\begin{align*}
\underset{\mathbf w_1, \ldots, \mathbf w_k}{\text{maximise}} &\quad \sum_{i=1}^k \mathbf w_i^\top \boldsymbol\Sigma \mathbf w_i \\
\text{subject to} &\quad \| \mathbf w_i \| = 1 &\quad i = 1, \ldots, k \\
&\quad \mathbf w_i^\top \mathbf w_j = 0 &\quad i \neq j
\end{align*}

- Subtle techinical point: not using greedy algorithms as in [sequential approach](#Sequential), which are not guaranteed to yeild optimal solutions
- However, same results as [sequential approach](#Sequential)

### Approximation error minimisation

Approximation error: $\displaystyle \mathbb E \| \mathbf x - \mathbf{Px} \|^2 = \mathbb E \| \mathbf x - \mathbf W_k \mathbf W_k^\top \mathbf x \|^2 = \mathbb E \| \mathbf x - \sum_{i=1}^k \mathbf w_k \mathbf w_k^\top \mathbf x \|^2$
- $\underset{d\times k}{\mathbf W_k} = (\mathbf w_1, \ldots, \mathbf w_k)$, where $\mathbf w_i$ are orthogonal vectors spanning a $k$-dim subspace of $\mathbb R^d$

\begin{align*}
\underset{\mathbf w_1, \ldots, \mathbf w_k}{\text{minimise}} &\quad \mathbb E \| \mathbf x - \sum_{i=1}^k \mathbf w_k \mathbf w_k^\top \mathbf x \|^2 \\
\text{subject to} &\quad \| \mathbf w_i \| = 1 &\quad i = 1, \ldots, k \\
&\quad \mathbf w_i^\top \mathbf w_j = 0 &\quad i \neq j
\end{align*}

- Equivalent to [simultaneous variance maximisation](#Simultaneous)
- Optimal $\mathbf w_i$ are the first $k$ eigenvectors $\mathbf u_i$ of $\boldsymbol\Sigma$
- $\displaystyle \mathbb E \| \mathbf x - \sum_{i=1}^k \mathbf w_k \mathbf w_k^\top \mathbf x \|^2 = \sum_{i=k+1}^d \lambda_i$

### Low rank matrix approximation

#### Data matrix

\begin{align*}
\underset{\mathbf M}{\text{minimise}} &\quad \| \mathbf X - \mathbf M \|_F^2 = \sum_{ij} \left( (\mathbf X)_{ij} - (\mathbf M)_{ij} \right)^2 \\
\text{subject to} &\quad \mathrm{rank}(\mathbf M) = k &\quad k<r=\mathrm{rank}(\mathbf X)
\end{align*}

- Optimal solution: $\hat{\mathbf X} = \mathbf U_k \mathbf S_k \mathbf V_k^\top$ (truncated singular value decomposition)
- Singular values relate to eigenvalues: $\displaystyle \lambda_i = \frac{s_i^2}{n}$
- Right singular vectors $\mathbf v_i$ are eigenvectors and hence PC directions
- PC scores: $\mathbf z_i = \mathbf{Xv}_i = \mathbf{USV}^\top \mathbf{Ve}_i = \mathbf{USe}_i = s_i \mathbf u_i$

#### Sample covariance matrix

\begin{align*}
\underset{\mathbf M}{\text{minimise}} &\quad \| \boldsymbol\Sigma - \mathbf M \|_F \\
\text{subject to} &\quad \mathrm{rank}(\mathbf M) = k &\quad k<r=\mathrm{rank}(\mathbf X) \\
&\quad \mathbf M^\top = \mathbf M
\end{align*}

- Optimal solution: $\hat{\boldsymbol\Sigma} = \mathbf V_k \boldsymbol\Lambda_k \mathbf V_k^\top$ (eigendecomposition)

#### Gram matrix

1. Eigendecomposition: $\displaystyle \underset{n\times n}{\mathbf G} = \mathbf X \mathbf X^\top \overset{\text{SVD}}{=} (\mathbf{USV}^\top) (\mathbf{USV}^\top)^\top = \mathbf{USS}^\top \mathbf U^\top = \mathbf U \tilde{\boldsymbol\Lambda} \mathbf U^\top$
1. Approximate $\mathbf G$ like [sample cov](#Sample-covariance-matrix): $\displaystyle \hat{\mathbf G} = \sum_{i=1}^k s_i^2 \mathbf u_i \mathbf u_i^\top = \sum_{i=1}^k \mathbf z_i \mathbf z_i^\top$

### Probabilistic PCA

#### Model

\begin{align*}
    \underset{K \times 1}{\mathbf z} &\sim \mathcal N(\mathbf 0, \mathbf I_K) \\
    \underset{D \times 1}{\boldsymbol\varepsilon} &\sim \mathcal N(\mathbf 0, \sigma^2 \mathbf I_D) \\
    \underset{D \times 1}{\mathbf x} &= \underset{D \times K}{\mathbf W} \ \underset{K \times 1}{\mathbf z} + \underset{D \times 1}{\boldsymbol\mu} + \underset{D \times 1}{\boldsymbol\varepsilon}
\end{align*}

#### Distributions

\begin{align*}
p(\mathbf x | \mathbf z) &= \mathcal N(\mathbf x; \mathbf{Wz}+\boldsymbol\mu, \sigma^2 \mathbf I_D) \\
p(\mathbf x) &= \mathcal N(\mathbf x; \boldsymbol\mu, \mathbf{WW}^\top+\sigma^2 \mathbf I_D)
\end{align*}

#### Maximum likelihood

\begin{align*}
\mathbf W_{\text{ML}} &= \mathbf U_k (\boldsymbol\Lambda_k - \sigma^2 \mathbf I)^{1/2} \mathbf R \\
\sigma^2_{\text{ML}} &= \frac{1}{d-k} \sum_{i=k+1}^d \lambda_i
\end{align*}

- $\mathbf U_k$ are $k$ principal eigenvectors of $\hat{\boldsymbol\Sigma} = \mathrm{cov}(\mathbf X) = \frac{1}{n}\mathbf{XX}^\top$
- $\boldsymbol\Lambda_k$ is diagonal matrix with eighenvalues
- $\mathbf R$ is arbitrary orthogonal matrix, interpreted as a rotation in the latent space, indicating not unique solutions

#### Relation to PCA

$$p(\mathbf z | \mathbf x) = \mathcal N(\mathbf z; \mathbf M^{-1} \mathbf W^\top (\mathbf x - \boldsymbol\mu), \sigma^2 \mathbf M^{-1})$$

- $\mathbf M = \mathbf W^\top \mathbf W + \sigma^2 \mathbf I$
- Projection $\displaystyle \hat{\mathbf x} = \mathbf W_{\text{ML}} \mathbb E[\mathbf z|\mathbf x] = \mathbf W_{\text{ML}} \mathbf M^{-1}_{\text{ML}} \mathbf W^\top_{\text{ML}} \mathbf x \to \mathbf U_k \mathbf U_k \mathbf U_k^\top \mathbf x$ as $\sigma^2 \to 0$

## Dimensionality Reduction

### Linear

- Does not take manifold structure of data into account

#### Data points

1. Observed data $\tilde{\mathbf X} = (\tilde{\mathbf x}_1, \ldots, \tilde{\mathbf x}_n)$
1. Centre data $\mathbf X = \tilde{\mathbf X} \mathbf C_n$ [(Standardisation)](#Standardisation)
1. Compute PC scores $\mathbf X = \mathbf U_k^\top \mathbf X$, via computation of PC directions
1. Compute PC scores $\mathbf Z = \sqrt{\tilde{\boldsymbol\Lambda}_k} \mathbf V_k^\top$ directly from [Gram matrix](#Gram-matrix)

#### Inner products

$$\mathbf G = \mathbf X^\top \mathbf X = \mathbf C_n^\top \tilde{\mathbf X}^\top \tilde{\mathbf X} \mathbf C_n = C_n \tilde{\mathbf G} \mathbf C_n$$

#### Distances

$$\mathbf G = -\frac{1}{2} \mathbf C_n \boldsymbol\Delta \mathbf C_n$$

### Kernel PCA

**New data matrix:**

$$\boldsymbol\Phi = (\boldsymbol\phi_1, \ldots, \boldsymbol\phi_n) = (\phi(\mathbf x_1), \ldots, \phi(\mathbf x_n))$$

**Uncentred Gram matrix:**

$$(\tilde{\mathbf G})_{ij} = \boldsymbol\phi_i^\top \boldsymbol\phi_j = \phi(\mathbf x_1)^\top \phi(\mathbf x_j) = k(\mathbf x_i, \mathbf x_j)$$

**Example kernels:**
- Polynomial kernel: $k(\mathbf x, \mathbf x') = (\mathbf x^\top \mathbf x')^a$
- Gaussian kernel: $\displaystyle k(\mathbf x, \mathbf x') = \exp \left( -\frac{\| \mathbf x - \mathbf x' \|^2}{2\sigma^2} \right)$

### Multidimensional scaling

#### Metric MDS

$$\underset{\mathbf z_1, \ldots, \mathbf z_n}{\text{minimise}} \quad \sum_{i<j} w_{ij} ( \| \mathbf z_i - \mathbf z_j \| - \delta_{ij})^2$$

- $\delta_{ij}$ are dissimilarities measuring difference between two data items, e.g. Euclidean distances. 
- $w_{ij} \geq 0$ specified by user
- If $\displaystyle w_{ij} = \frac{1}{\delta_{ij}}$, Sammon nonlinear mapping, emphasising the faithful representation of small dissimilarities

#### Nonmetric MDS

$$\underset{\mathbf z_1, \ldots, \mathbf z_n, f}{\text{minimise}} \quad \sum_{i<j} w_{ij} \left( \| \mathbf z_i - \mathbf z_j \| - f(\delta_{ij}) \right)^2$$

- Only relation between $\delta_{ij}$ matter
- $f$ is monotonic

#### Classical MDS

- $\boldsymbol\Delta$ is distance matrix between the unknown vectors, not necessarily postive semidefinite
- Follow [this steps](#Distances)
- Choose small $k$ to avoid negative eigenvalues

- Alternatively, approximate negative definite $\boldsymbol\Delta$ by

\begin{align*}
\underset{\mathbf M}{\text{minimise}} &\quad \left\| (-\frac{1}{2} \mathbf C_n \boldsymbol\Delta \mathbf C_n) - \mathbf M^\top \mathbf M \right\|_F \\
\text{subject to} &\quad \mathrm{rank}(\mathbf M^\top \mathbf M) = k
\end{align*}

### Isomap

- Use geodesic distances representing the shortest paths along the curved surface


1. Find the neighbours of each data point in high-dimensional data
1. Compute the geodesic pairwise distances between all points
1. Embed the data via MDS so as to preserve these distances

## Predictive Modelling and Generalisation

### Losses

#### Prediction loss

$$\mathcal J(h) = \mathbb E_{p(\hat y, y)} [\mathcal L(\hat y, y)] = \mathbb E_{p(\mathbf x, y)} [\mathcal L(h(\mathbf x), y)]$$

#### Training loss

$$J_{\boldsymbol\lambda}^* = \min_{\boldsymbol\theta} J_{\boldsymbol\lambda}(\boldsymbol\theta) = \frac{1}{n} \sum_{i=1}^n L(h_{\boldsymbol\lambda}(\mathbf x_i; \boldsymbol\theta), y_i)$$

### Generalisation performance

#### Generalisation loss

**For prediction functions:**

$$\mathcal J(\hat h) = \mathbb E_{p(\mathbf x, y)} \left[ \mathcal L(\hat h(x), y) \right]$$

**For algorithms:**

$$\bar{\mathcal J}(\mathcal A) = \mathbb E_{p(\mathcal D^{\text{train}})} \left[ \mathcal J (\hat h) \right] = \mathbb E_{p(\mathcal D^{\text{train}})} \left[ \mathcal J(\mathcal A(\mathcal D^{\text{train}})) \right]$$

#### Overfitting and underfitting

Overfitting: reduce complexity of model $\to$ reduce prediction loss

Underfitting: choose more flexible model $\to$ reduce prediction loss

$$\underset{\boldsymbol\theta}{\text{minimise}} \quad \mathcal J_{\boldsymbol\lambda}(\boldsymbol\theta) + \lambda_{\text{reg}} R(\boldsymbol\theta)$$

#### Hyperparameter selection

Consider both complexity and training size of models

### Estimating the generalisation performance

#### Methods

##### Hold-out approach

Prediction function:

$$\hat h = \mathcal A(\mathcal D^{\text{train}})$$

Prediction loss on $\tilde{\mathcal D}$ (validation/test set):

$$\hat{\mathcal J}(\hat h; \tilde{\mathcal D}) = \frac{1}{\tilde n} \sum_{i=1}^{\tilde n} \mathcal L(\hat h(\tilde{\mathbf x_i}), \tilde y_i)$$

- Common split ratios: 6:4, 7:3 or 8:2
- Increase ratio if the number of hyperparameter is large
- Split randomly
- Stratification (classes are in the same proportions in both sets)
- Drawback (use cross-validation to avoid)

##### Cross-validation

$K$-fold:

$$\mathcal D_k^{\text{train}} = \bigcup_{i \neq k} \mathcal D_i \quad \mathcal D_k^{\text{val}} = \mathcal D_k$$

Prediction function and loss for the $k$-th fold:

$$\hat h_k = \mathcal A(\mathcal D_k^{\text{train}}) \quad \hat{\mathcal J_k} = \hat{\mathcal J}(\hat h_k; \mathcal D_k^{\text{val}})$$

Cross-validation score:

$$\mathrm{CV} = \frac{1}{K} \sum_{i=1}^K \hat{\mathcal J_k} = \frac{1}{K} \sum_{i=1}^K \hat{\mathcal J} \left( \mathcal A(\mathcal D_k^{\text{train}}); \mathcal D_k^{\text{val}} \right) = \hat{\bar{\mathcal J}}(\mathcal A)$$

Estimated variance of CV score:

$$\begin{cases}
\displaystyle \mathrm{Var}(\mathrm{CV}) \approx \frac{1}{K} \mathrm{Var}(\hat{\mathcal J}) \\
\displaystyle \mathrm{Var}(\hat{\mathcal J}) \approx \frac{1}{K} \sum_{k=1}^K (\hat{\mathcal J_k} - \mathrm{CV})^2
\end{cases}$$

indeed, $\displaystyle \hat{\mathrm{Var}}(\mathrm{CV}) = \frac{1}{K(K-1)} \sum_{k=1}^K (\hat{\mathcal J_k} - \mathrm{CV})^2$

- LOOCV: generally expensive, but possible in e.g. Gaussian time series and spatial models

#### Hyperparameter selection and performance evaluation

##### Two times hold-out

1. Split off test set $\mathcal D^{\text{test}}$ for estimating the performance of final prediction function
1. Split remaining data into $\mathcal D^{\text{train}}$ and $\mathcal D^{\text{val}}$
1. Tune parameters $\boldsymbol\lambda$ in an algorithm $\displaystyle \hat h_{\boldsymbol\lambda} = \mathcal A_{\boldsymbol\lambda}(\mathcal D^{\text{train}})$
1. Compute prediction loss $\displaystyle \mathrm{PL}(\boldsymbol\lambda) = \hat{\mathcal J}(\hat h_{\boldsymbol\lambda}; \mathcal D^{\text{val}})$ and choose $\displaystyle \hat{\boldsymbol\lambda} = \underset{\boldsymbol\lambda}{\arg\min} \mathrm{PL}(\boldsymbol\lambda)$
1. Re-estimate on union of training and validation data $\displaystyle \hat h = \mathcal A_{\hat{\boldsymbol\lambda}} (\mathcal D^{\text{train}} \cup \mathcal D^{\text{val}})$
1. Compute prediction loss on test data $\displaystyle \hat{\mathcal J} = \hat{\mathcal J}(\hat h; \mathcal D^{\text{test}})$
1. Re-estimate on all data $\displaystyle \hat h_{\boldsymbol\lambda} = \mathcal A_{\hat{\boldsymbol\lambda}} (\mathcal D)$


- Re-estimation sometimes needs to be skipped
- Optimisation methods over $\boldsymbol\lambda$: grid search, random search, Bayesian optimisation

##### Cross-validation and hold-out

1. Split off test set $\mathcal D^{\text{test}}$ for estimating the performance of final prediction function
1. Compute CV score on remaining data $\mathcal D^{\text{train}}$: $\displaystyle \mathrm{EPL}(\boldsymbol\lambda) = \mathrm{CV} = \hat{\bar{\mathcal J}} (\mathcal A_{\boldsymbol\lambda})$
1. Choose $\displaystyle \hat{\boldsymbol\lambda} = \underset{\boldsymbol\lambda}{\arg\min} \mathrm{EPL}(\boldsymbol\lambda)$ OR simplest model and $\displaystyle \mathrm{EPL}(\hat{\boldsymbol\lambda}) \in \left[\min \mathrm{CV} \pm \sqrt{\mathrm{Var}(\min \mathrm{CV})}\right]$
1. Re-estimate on whole training data $\displaystyle \hat h = \mathcal A_{\hat{\boldsymbol\lambda}}(\mathcal D^{\text{train}})$
1. Compute prediction loss on test data $\displaystyle \hat{\mathcal J} = \hat{\mathcal J}(\hat h; \mathcal D^{\text{test}})$
1. Re-estimate on all data $\displaystyle \hat h_{\boldsymbol\lambda} = \mathcal A_{\hat{\boldsymbol\lambda}} (\mathcal D)$


- Minimal CV score is an optimistic estimate of prediction loss

### Loss functions

#### In regression

- Square loss: $\displaystyle L(\hat y, y) = \frac{1}{2} (\hat y - y)^2$
- Absolute loss: $\displaystyle L(\hat y, y) = |\hat y - y|$ (more robust than square loss because not grow quickly, but not differentiable)
- Huber loss: $\displaystyle L(\hat y, y) = \begin{cases}
\displaystyle \frac{1}{2} (\hat y - y)^2 & \text{if } |\hat y - y| < \delta \\
\displaystyle \delta |y - \hat y| - \frac{1}{2}\delta^2 & \text{otherwise}
\end{cases}$ (combimes good properties of square and absolute loss)

#### In classification

##### Non-differentiable

$$\mathbf L_{\hat y, y} = \begin{pmatrix}
L(1,1) & L(1,2) & \cdots & L(1,K) \\
L(2,1) & L(2,2) & \cdots & L(2,K) \\
\vdots & \vdots & & \vdots \\
L(K,1) & L(K,2) & \cdots & L(K,K)
\end{pmatrix}$$

**Zero-one loss:**

$$L(i,j) = \begin{cases}
1 & i \neq j \\
0 & i = j
\end{cases}$$

- Expectation: $\displaystyle \mathcal J(h) = \sum_{i \neq j} p(i,j) = \mathbb P(i\neq j)$ (misclassification/error rate)

**Binary classification:**

| $\hat y$ | $y$ | event | joint probability $p(\hat y, y)$ | shorthand notation | rate | conditional probability $p(\hat y|y)$ |
|---|---|---|---|---|---|---|
| 1 | 1 | True Positive | $\mathbb P(\hat y=1, y=1)$ | $p(1,1)$ | TP rate<br>sensitivity, hit rate, recall | $\mathbb P(\hat y=1 | y=1)$ |
| 1 | -1 | False Positive | $\mathbb P(\hat y=1, y=-1)$ | $p(1,-1)$ | FP rate<br>type 1 error, fall-out | $\mathbb P(\hat y=1 | y=-1)$ |
| -1 | 1 | False Negative | $\mathbb P(\hat y=-1, y=1)$ | $p(-1,1)$ | FN rate<br>type 2 error | $\mathbb P(\hat y=-1 | y=1)$ |
| -1 | -1 | True Negative | $\mathbb P(\hat y=-1, y=-1)$ | $p(-1, -1)$ | TN rate<br>specificity | $\mathbb P(\hat y=-1 | y=-1)$ |

**Loss function penalising FP and FN rates:**

$$\mathbf L_{\hat y, y} = \begin{pmatrix}
0 & \frac{1}{\mathbb P(y=1)} \\
\frac{1}{\mathbb P(y=-1)} & 0 \\
\end{pmatrix}$$

- Expectation: $\displaystyle \mathcal J(h) = \sum_{i,j} L(i,j) p(i,j) = \mathrm{FPR} + \mathrm{FNR}$
- Not meaningful because optimal solution is $\hat y=-1$: $\mathrm{TPR}=0$

**Reciver operating characteristic (ROC):**

- TPR vs FPR
- Perfect classifier at upper-left corner

##### Differentiable

Assume $\hat y(\mathbf x) = \mathrm{sign}(h(\mathbf x))$, correct classification equivalent to margin $yh(\mathbf x) >0$

- Zero-one loss: $\displaystyle L(h(\mathbf x), y) = \begin{cases}
1 & \text{if } yh(\mathbf x) < 0 \\
0 & \text{otherwise}
\end{cases}$
- Square loss: $L(h(\mathbf x), y) = (1-yh(\mathbf x))^2$ (sensitive to outliers and penalises large margins)
- Log loss: $L(h(\mathbf x), y) = \log (1+\exp (-yh(\mathbf x)))$ (equivalent to maximising log-likelihood in logistic regression)
- Exponential loss: $L(h(\mathbf x), y) = \exp(-yh(\mathbf x))$
- Hinge loss: $L(h(\mathbf x),y) = \max(0, 1-yh(\mathbf x))$
- Squared hinge loss: $L(h(\mathbf x),y) = \max(0, 1-yh(\mathbf x))^2$ (differentiable everywhere)
- Huberised squared hinge loss: $\displaystyle L(h(\mathbf x),y) = \begin{cases}
-4yh(\mathbf x) & \text{if } yh(\mathbf x) < -1 \\
\max(0, 1-yh(\mathbf x))^2 & \text{otherwise}
\end{cases}$

**Authors:** s1680642

**Licensing:** <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.